# Brick-Sort

## To implement sorting on a large array by parallel version of Bubble sort algorithm(Brick sort) and benchmarking it with traditional serial version

In [2]:
using Pkg
Pkg.add("CuArrays")
Pkg.add("CUDAnative")
Pkg.add("CUDAdrv")
Pkg.add("StaticArrays")
Pkg.add("BenchmarkTools")
Pkg.add("Test")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [be33ccc6] + CUDAnative v2.2.0
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [c5f51814] + CUDAdrv v3.0.1
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [90137ffa] + StaticArrays v0.11.0
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
 Installed BenchmarkTools ─ v0.4.2
  Updating `~/.julia/environments/v1.0/Project.toml`
  [6e4b80f9] + BenchmarkTools v0.4.2
 

In [0]:
using CUDAnative, CUDAdrv, CuArrays, StaticArrays, BenchmarkTools, Test

In [3]:
function Bubble_sort!(h_in)
  
  for i=1:size(h_in,1)
    for j=i:size(h_in,1)
      if(h_in[i] > h_in[j])
        temp = h_in[i]
        h_in[i] = h_in[j]
        h_in[j] = temp
      end
    end
  end
  
end

Bubble_sort! (generic function with 1 method)

In [4]:
function Brick_sort!(d_in)

  idx = (blockIdx().x - 1) * blockDim().x + threadIdx().x
  
  if(idx <= div(size(d_in,1),2))
    
    right = 2*idx+1
    mid = 2*idx
    left = 2*idx-1
    
    for i=1:size(d_in,1)
      if(i%2==0)
        if(d_in[left] > d_in[mid])
          temp = d_in[left]
          d_in[left] = d_in[mid]
          d_in[mid] = temp
        end
      
      else
        if(right <= size(d_in,1) && d_in[mid] > d_in[right])
          temp = d_in[right]
          d_in[right] = d_in[mid]
          d_in[mid] = temp
        end
      end
      
      sync_threads()
    end
  end

return 
end

Brick_sort! (generic function with 1 method)

In [7]:
N = 2^10
h_in = rand(N)
d_in = CuArray(h_in)

1024-element CuArray{Float64,1}:
 0.1902493749243972  
 0.7821302651287456  
 0.07280937099217755 
 0.012185572215902551
 0.9262451891956638  
 0.8556324744097903  
 0.36803655153967907 
 0.09267199541584659 
 0.7342172087957521  
 0.8927330179376816  
 0.740019559288948   
 0.42934853585859645 
 0.28768755187961026 
 ⋮                   
 0.4837236638107367  
 0.01743771796902638 
 0.05329951858566795 
 0.525737004379367   
 0.17820009458989738 
 0.7387487333780158  
 0.8565021470777872  
 0.6340072944460173  
 0.889442887893287   
 0.9616830460119639  
 0.4496037222073199  
 0.4999690274500266  

In [0]:
Bubble_sort!(h_in)

In [0]:
@cuda blocks = Int(ceil(N/2^10)) threads = 2^9 Brick_sort!(d_in)

In [9]:
h_out = Array(d_in)

1024-element Array{Float64,1}:
 0.0006857128740640483
 0.000873221267461588 
 0.002371801785977734 
 0.0026102539807977276
 0.003577637411283474 
 0.0036537385431225644
 0.0036778820996357098
 0.0038679654661974983
 0.004982500039236948 
 0.005913171242535098 
 0.006200990920913085 
 0.008872071356957667 
 0.009019536660738181 
 ⋮                    
 0.9877403010629404   
 0.989601703270774    
 0.9901753203439232   
 0.9922037018809802   
 0.9927558726233445   
 0.9936314089748262   
 0.9944949969162256   
 0.9961412976095607   
 0.9973420389820165   
 0.9978294214694181   
 0.9996623537563409   
 0.9997819433523107   

In [11]:
@test isapprox(h_in,h_out)

Test Passed

In [14]:
@benchmark Bubble_sort!(h_in)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     607.592 μs (0.00% GC)
  median time:      656.882 μs (0.00% GC)
  mean time:        666.588 μs (0.00% GC)
  maximum time:     1.193 ms (0.00% GC)
  --------------
  samples:          7483
  evals/sample:     1

In [15]:
@benchmark @cuda blocks = Int(ceil(N/2^10)) threads = 2^9 Brick_sort!(d_in)

BenchmarkTools.Trial: 
  memory estimate:  880 bytes
  allocs estimate:  32
  --------------
  minimum time:     8.959 μs (0.00% GC)
  median time:      484.666 μs (0.00% GC)
  mean time:        439.203 μs (0.00% GC)
  maximum time:     3.036 ms (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [12]:
@btime Bubble_sort!(h_in)

  607.549 μs (0 allocations: 0 bytes)


In [13]:
@btime @cuda blocks = Int(ceil(N/2^10)) threads = 2^9 Brick_sort!(d_in)

  9.053 μs (32 allocations: 880 bytes)


## Parallel implementation is about 70 times faster than serial one for input upto 10^3 elements
### @benchmark runs the code several number of times which results in the above output(still parallel code is faster than sequential)